# Welcome to Titanic EDA and Modelling

## Data description:

- **survival:**	Survival (0 = No, 1 = Yes).<br>
- **pclass**	Ticket class. (1 = 1st, 2 = 2nd, 3 = 3rd)<br>	
- **sex:**	Sex.<br>	
- **Age:**	Age in years.<br>	
- **sibsp:**	# of siblings / spouses aboard the Titanic.<br>	
- **parch:**	# of parents / children aboard the Titanic.<br>	
- **ticket:**	Ticket number.<br>	
- **fare:**	Passenger fare.<br>	
- **cabin:**	Cabin number.<br>	
- **embarked:**	Port of Embarkation. (C, Q, S)<br>	

## Necessary libraries

In [35]:
#Data processing
import pandas as pd
import numpy as np

#sklearn Pipelines
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

#models
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

#split data
from sklearn.model_selection import train_test_split

#Feature engineering
from sklearn.inspection import permutation_importance

#evaluation and model selection
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import RandomizedSearchCV

#save model
from joblib import dump

In [2]:
RANDOM_SEED = 42

In [3]:
titanic_data = pd.read_csv("../data/train.csv")
titanic_data.info()
titanic_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### Columns to drop:<br>
**PassengerId**, **Name** and **Ticket** don´t have any relationship to the survive probability <br>
The values in **Cabin** are mostly null


In [4]:
titanic_data.drop(['Cabin', 'PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

#### Create new columns that could improve prediction

In [5]:
titanic_data['FamilySize'] = titanic_data['SibSp'] + titanic_data['Parch']
titanic_data['IsAlone'] = 0
titanic_data.loc[titanic_data['FamilySize'] == 0, 'IsAlone'] = 1

#### Categorize the columns types for preprocessing

In [6]:
ordinal_attributes = ['Sex', 'IsAlone']
numeric_features = ['Parch', 'Pclass', 'SibSp', 'Fare', 'Age']
categorical_features = ['Embarked', 'FamilySize']
target = 'Survived'

print(f"Numeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")
print(f"Ordinal features: {ordinal_attributes}")

Numeric features: ['Parch', 'Pclass', 'SibSp', 'Fare', 'Age']
Categorical features: ['Embarked', 'FamilySize']
Ordinal features: ['Sex', 'IsAlone']


In [7]:
y = titanic_data[target]
X = titanic_data.drop(columns=[target]) 

#### Create preprocessing pipelines

In [8]:
num_processing_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy="median")),
                         ('std_scaler', StandardScaler()),
])

cat_processing_pipeline = Pipeline([
                              ('imputer', SimpleImputer(strategy="most_frequent")),
                              ('cat', OneHotEncoder()),
])

data_processing_pipeline = ColumnTransformer([
                                   ("num", num_processing_pipeline, numeric_features),
                                   ("ord", OrdinalEncoder(), ordinal_attributes),
                                   ("cat", cat_processing_pipeline, categorical_features),
])

In [9]:
X_processed = data_processing_pipeline.fit_transform(X,y)

In [10]:
categorical_features_columns = []
for col in categorical_features:
    for x in range(0, len(titanic_data[col].dropna().unique())):
        categorical_features_columns.extend([col + str(x)])

In [11]:
new_columns =  numeric_features + ordinal_attributes + categorical_features_columns
X_processed = pd.DataFrame(X_processed, columns=new_columns)
X_processed.head()

,Parch,Pclass,SibSp,Fare,Age,Sex,IsAlone,Embarked0,Embarked1,Embarked2,FamilySize0,FamilySize1,FamilySize2,FamilySize3,FamilySize4,FamilySize5,FamilySize6,FamilySize7,FamilySize8
0,-0.473674,0.827377,0.432793,-0.502445,-0.565736,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.473674,-1.566107,0.432793,0.786845,0.663861,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.473674,0.827377,-0.474545,-0.488854,-0.258337,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.473674,-1.566107,0.432793,0.420730,0.433312,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.473674,0.827377,-0.474545,-0.486337,0.433312,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Select most important features using permutation importance tecnique

#### Calculate importance scores and add a random column as baseline

In [12]:
def train_feature_selection_model(X, y, verbose=True):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=RANDOM_SEED)

    # Set a random seed for reproducibility
    random_seed = 42
    np.random.seed(random_seed)
    X_train["random_feature"] = np.random.randn(X_train.shape[0]) # add a random feature
    X_test["random_feature"] = np.random.randn(X_test.shape[0]) # add a random feature

    # Initialize a Random Forest classifier
    model = RandomForestClassifier(random_state=RANDOM_SEED)

    # Train the model
    model.fit(X_train, y_train) 

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate the accuracy of the model
    accuracy = accuracy_score(y_test, y_pred)
    if verbose: print(f"Model Accuracy: {accuracy:.2f}")

    return model, X_test, y_test, X_train, y_train

def calculate_permutation_importance(model, X_test, y_test, verbose=True):
    # Calculate permutation importance
    perm_importance = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=RANDOM_SEED)

    if verbose:
        for i, feature in enumerate(X_test.columns):
            print(f"{feature}: {perm_importance.importances_mean[i]:.4f}")
    
    return perm_importance

model, X_test, y_test, X_train, y_train = train_feature_selection_model(X_processed, y)
perm_importance = calculate_permutation_importance(model, X_test, y_test)

Model Accuracy: 0.80
Parch: -0.0010
Pclass: 0.0332
SibSp: -0.0027
Fare: 0.0200
Age: 0.0237
Sex: 0.1708
IsAlone: 0.0003
Embarked0: -0.0183
Embarked1: -0.0010
Embarked2: -0.0197
FamilySize0: -0.0024
FamilySize1: -0.0010
FamilySize2: 0.0034
FamilySize3: 0.0031
FamilySize4: -0.0003
FamilySize5: 0.0000
FamilySize6: -0.0003
FamilySize7: 0.0000
FamilySize8: 0.0000
random_feature: 0.0020


#### Select columns with importance score above the random column score

In [13]:
def select_features(perm_importance, X_train, X_test, verbose=True):

    # Get feature importance scores and indices
    feature_scores = perm_importance.importances_mean
    feature_indices = [i for i in range(len(X_test.columns))]

    # Select features with score above the random column score

    random_feature_importance = perm_importance.importances_mean[X_train.columns.get_loc("random_feature")]
    selected_features = [X_test.columns[i] for i in feature_indices if feature_scores[i] > random_feature_importance]

    # Print selected features
    if verbose:
        print("Selected Features:")
        for feature in selected_features:
            print(feature)
    
    return selected_features

def test_selected_features(selected_features, X_train, X_test, y_train, y_test, verbose=True):

    # Train a new model using only the selected features
    selected_X_train = X_train[selected_features]
    selected_X_test = X_test[selected_features]

    selected_model = RandomForestClassifier(random_state=42)
    selected_model.fit(selected_X_train, y_train)

    # Make predictions on the test set using the selected model
    selected_y_pred = selected_model.predict(selected_X_test)

    # Calculate the accuracy of the selected model
    selected_accuracy = accuracy_score(y_test, selected_y_pred)
    if verbose: print(f"Selected Features Model Accuracy: {selected_accuracy:.2f}")

selected_features = select_features(perm_importance, X_train, X_test)
test_selected_features(selected_features, X_train, X_test, y_train, y_test)


Selected Features:
Pclass
Fare
Age
Sex
FamilySize2
FamilySize3
Selected Features Model Accuracy: 0.79


The model accuracy barely changed using only the selected features

#### Now let's create the pipeline for feature selection

In [14]:
class FeatureSelection(BaseEstimator, TransformerMixin):

  def __init__(self, columns, verbose=False):
    super().__init__()
    self.columns = columns
    self.verbose = verbose
  
  def fit(self, X, y=None):
    X_df = self.to_dataframe(X)
    model, X_test, y_test, X_train, y_train = train_feature_selection_model(X_df, y, self.verbose)
    perm_importance = calculate_permutation_importance(model, X_test, y_test, self.verbose)
    selected_features = select_features(perm_importance, X_train, X_test, self.verbose)
    test_selected_features(selected_features, X_train, X_test, y_train, y_test, self.verbose)
    self.selected_features = selected_features
    return self

  def to_dataframe(self, X):
    X_processed = pd.DataFrame(X, columns=self.columns)
    return X_processed


  def transform(self, X):
    X_df = self.to_dataframe(X)
    X_df = X_df[self.selected_features]
    return X_df

In [15]:
processing_feature_selection_pipeline = Pipeline([
                                            ("data_processing", data_processing_pipeline),
                                            ("FeatureSelection", FeatureSelection(columns=new_columns))
                                            ])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=RANDOM_SEED)

In [17]:
X_train_processed = processing_feature_selection_pipeline.fit_transform(X_train, y_train)

In [18]:
X_test_processed = processing_feature_selection_pipeline.transform(X_test)

## Train, evaluate and choose model

In [19]:
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
    'random_state': [42]
}

param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3],
    'subsample': [0.8, 0.9, 1.0],
    'random_state': [42]
}

param_grid_knn = {
    'n_neighbors': np.arange(1, 21),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Perform RandomizedSearchCV for each classifier
rf_random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_grid_rf, n_iter=15, cv=5, verbose=1, random_state=42, n_jobs=-1)
rf_random_search.fit(X_train_processed, y_train)

gb_random_search = RandomizedSearchCV(GradientBoostingClassifier(), param_distributions=param_grid_gb, n_iter=15, cv=5, verbose=1, random_state=42, n_jobs=-1)
gb_random_search.fit(X_train_processed, y_train)

knn_random_search = RandomizedSearchCV(KNeighborsClassifier(), param_distributions=param_grid_knn, n_iter=15, cv=5, verbose=1, random_state=42, n_jobs=-1)
knn_random_search.fit(X_train_processed, y_train)

# Print best parameters for each classifier
print("Random Forest - Best Parameters:", rf_random_search.best_params_)
print("Gradient Boosting - Best Parameters:", gb_random_search.best_params_)
print("K-Nearest Neighbors - Best Parameters:", knn_random_search.best_params_)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


Fitting 5 folds for each of 15 candidates, totalling 75 fits
Fitting 5 folds for each of 15 candidates, totalling 75 fits
Random Forest - Best Parameters: {'random_state': 42, 'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': True}
Gradient Boosting - Best Parameters: {'subsample': 0.9, 'random_state': 42, 'n_estimators': 200, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_depth': 4, 'learning_rate': 0.01}
K-Nearest Neighbors - Best Parameters: {'weights': 'uniform', 'n_neighbors': 12, 'metric': 'euclidean'}


In [20]:
# Evaluate the best models on the test set
rf_best = rf_random_search.best_estimator_
gb_best = gb_random_search.best_estimator_
knn_best = knn_random_search.best_estimator_

rf_accuracy = rf_best.score(X_test_processed, y_test)
gb_accuracy = gb_best.score(X_test_processed, y_test)
knn_accuracy = knn_best.score(X_test_processed, y_test)

print("Random Forest - Test Accuracy:", rf_accuracy)
print("Gradient Boosting - Test Accuracy:", gb_accuracy)
print("K-Nearest Neighbors - Test Accuracy:", knn_accuracy)

Random Forest - Test Accuracy: 0.8067796610169492
Gradient Boosting - Test Accuracy: 0.7932203389830509
K-Nearest Neighbors - Test Accuracy: 0.7728813559322034


The best model is Random Forest

### Final Step: Create the full preprocessing, feature selection and model training pipeline

In [22]:
full_pipeline_rf = Pipeline([
                ('preparation', processing_feature_selection_pipeline),
                ('model', rf_random_search)
            ])

full_pipeline_gb = Pipeline([
                ('preparation', processing_feature_selection_pipeline),
                ('model', gb_random_search)
            ])

full_pipeline_knn = Pipeline([
                ('preparation', processing_feature_selection_pipeline),
                ('model', knn_random_search)
            ])

In [31]:
display(full_pipeline_rf.fit(X_train, y_train))
display(full_pipeline_gb.fit(X_train, y_train))
display(full_pipeline_knn.fit(X_train, y_train))

Fitting 5 folds for each of 15 candidates, totalling 75 fits


Pipeline(steps=[('preparation',
                 Pipeline(steps=[('data_processing',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='median')),
                                                                                   ('std_scaler',
                                                                                    StandardScaler())]),
                                                                   ['Parch',
                                                                    'Pclass',
                                                                    'SibSp',
                                                                    'Fare',
                                                                    'Age']),
                                                                  ('ord',
                                                                   OrdinalEncoder(),
                                                                   ['Sex',
                                                                    'IsAlone']),
                                                                  ('cat',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='most_fre...
                                                            'FamilySize7',
                                                            'FamilySize8']))])),
                ('model',
                 RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                                    n_iter=15, n_jobs=-1,
                                    param_distributions={'bootstrap': [True,
                                                                       False],
                                                         'max_depth': [None, 10,
                                                                       20],
                                                         'max_features': ['sqrt',
                                                                          'log2'],
                                                         'min_samples_leaf': [1,
                                                                              2,
                                                                              4],
                                                         'min_samples_split': [2,
                                                                               5,
                                                                               10],
                                                         'n_estimators': [100,
                                                                          200,
                                                                          300],
                                                         'random_state': [42]},
                                    random_state=42, verbose=1))])

Fitting 5 folds for each of 15 candidates, totalling 75 fits


Pipeline(steps=[('preparation',
                 Pipeline(steps=[('data_processing',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='median')),
                                                                                   ('std_scaler',
                                                                                    StandardScaler())]),
                                                                   ['Parch',
                                                                    'Pclass',
                                                                    'SibSp',
                                                                    'Fare',
                                                                    'Age']),
                                                                  ('ord',
                                                                   OrdinalEncoder(),
                                                                   ['Sex',
                                                                    'IsAlone']),
                                                                  ('cat',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='most_fre...
                                                            'FamilySize7',
                                                            'FamilySize8']))])),
                ('model',
                 RandomizedSearchCV(cv=5,
                                    estimator=GradientBoostingClassifier(),
                                    n_iter=15, n_jobs=-1,
                                    param_distributions={'learning_rate': [0.01,
                                                                           0.1,
                                                                           0.2],
                                                         'max_depth': [3, 4, 5],
                                                         'min_samples_leaf': [1,
                                                                              2,
                                                                              3],
                                                         'min_samples_split': [2,
                                                                               3,
                                                                               4],
                                                         'n_estimators': [100,
                                                                          200,
                                                                          300],
                                                         'random_state': [42],
                                                         'subsample': [0.8, 0.9,
                                                                       1.0]},
                                    random_state=42, verbose=1))])

Fitting 5 folds for each of 15 candidates, totalling 75 fits


Pipeline(steps=[('preparation',
                 Pipeline(steps=[('data_processing',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='median')),
                                                                                   ('std_scaler',
                                                                                    StandardScaler())]),
                                                                   ['Parch',
                                                                    'Pclass',
                                                                    'SibSp',
                                                                    'Fare',
                                                                    'Age']),
                                                                  ('ord',
                                                                   OrdinalEncoder(),
                                                                   ['Sex',
                                                                    'IsAlone']),
                                                                  ('cat',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='most_fre...
                                                            'FamilySize4',
                                                            'FamilySize5',
                                                            'FamilySize6',
                                                            'FamilySize7',
                                                            'FamilySize8']))])),
                ('model',
                 RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(),
                                    n_iter=15, n_jobs=-1,
                                    param_distributions={'metric': ['euclidean',
                                                                    'manhattan'],
                                                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20]),
                                                         'weights': ['uniform',
                                                                     'distance']},
                                    random_state=42, verbose=1))])

In [24]:
rf_accuracy = full_pipeline_rf.score(X_test, y_test)
gb_accuracy = full_pipeline_gb.score(X_test, y_test)
knn_accuracy = full_pipeline_knn.score(X_test, y_test)

print("Random Forest - Test Accuracy:", rf_accuracy)
print("Gradient Boosting - Test Accuracy:", gb_accuracy)
print("K-Nearest Neighbors - Test Accuracy:", knn_accuracy)

Random Forest - Test Accuracy: 0.8067796610169492
Gradient Boosting - Test Accuracy: 0.7932203389830509
K-Nearest Neighbors - Test Accuracy: 0.7728813559322034


In [32]:
# Determine the best model
best_model = None
best_accuracy = -1

if rf_accuracy > best_accuracy:
    best_accuracy = rf_accuracy
    best_pipeline = full_pipeline_rf
    best_model = "Random Forest"

if gb_accuracy > best_accuracy:
    best_accuracy = gb_accuracy
    best_pipeline = full_pipeline_gb
    best_model = "Gradient Boosting"

if knn_accuracy > best_accuracy:
    best_accuracy = knn_accuracy
    best_pipeline = full_pipeline_knn
    best_model = "K-Nearest Neighbors"

print("Best Model:", best_model)
print("Best Test Accuracy:", best_accuracy)

Best Model: Random Forest
Best Test Accuracy: 0.8067796610169492


#### Create the model registry

In [39]:
dump(best_pipeline, "../models/best_model.pkl")

['../models/best_model.pkl']

In [45]:
!dvc remote add -d model_remote s3://dvc-mle-test/model/ --force

Setting 'model_remote' as a default remote.


In [47]:
!dvc add '../models/best_model.pkl' --to-remote -r model_remote
!dvc push

⠋ Checking graph                                       core>
!
  0%|          |best_model.pkl                 0.00/1.70M [00:00<?,        ?B/s]
  0%|          |best_model.pkl                 0.00/1.70M [00:00<?,        ?B/s]
  0% Transferring to s3://dvc-mle-test/model/files/md5| |0/1 [00:00<?,     ?file
!
  0%|          |dvc-mle-test/model/files/md5/3a/5c50.00/? [00:00<?,        ?B/s]
  0%|          |dvc-mle-test/model/files/md5/3a0.00/1.70M [00:00<?,        ?B/s]
100% Transferring to s3://dvc-mle-test/model/files/md5|█|1/1 [00:02<00:00,  2.15
                                                                                
To track the changes with git, run:

	git add ../models/best_model.pkl.dvc

To enable auto staging, run:

	dvc config core.autostage true
  0% Pushing to s3://dvc-mle-test/model/files/md5|   |0/2 [00:00<?,     ?file/s]
!
  0%|          |/home/santiago/rappi_mle_test/Titan0.00/? [00:00<?,        ?B/s]
  0%|          |/home/santiago/rappi_mle_test/T0.00/28.8k [00:00<?

Now we can always download the last version of the model using

In [48]:
!dvc pull -r model_remote

  0% Fetching from s3|                               |0/1 [00:00<?,     ?file/s]
!
  0%|          |dvc-mle-test/model/files/md5/3a/5c50.00/? [00:00<?,        ?B/s]
  0%|          |dvc-mle-test/model/files/md5/3a0.00/1.70M [00:00<?,        ?B/s]
  4%|▍         |dvc-mle-test/model/files/m67.5k/1.70M [00:00<00:04,     389kB/s]
 16%|█▌        |dvc-mle-test/model/files/md271k/1.70M [00:00<00:01,     830kB/s]
 38%|███▊      |dvc-mle-test/model/files/md662k/1.70M [00:00<00:00,    1.76MB/s]
 51%|█████     |dvc-mle-test/model/files/md883k/1.70M [00:00<00:00,    1.75MB/s]
 71%|███████▏  |dvc-mle-test/model/files/m1.21M/1.70M [00:00<00:00,    2.28MB/s]
 77%|███████▋  |dvc-mle-test/model/files/m1.31M/1.70M [00:00<00:00,    1.66MB/s]
100% Fetching from s3|███████████████████████████|1/1 [00:01<00:00,  1.13s/file]
Checkout                                              |0.00 [00:00,     ?file/s]
!
  0%|          |/home/santiago/rappi_mle_test/Titan0.00/? [00:00<?,        ?B/s]
  0%|          |/home/sa